In [8]:
# imports relevant libraries used for building the KNN
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score , recall_score

In [9]:
"""The function pads all spectrograms to make them uniform 
"""
def padder(list):
    max_check = 0
    for input in list:
        i,j = input.shape
        if max_check < j:
            max_check = j
            
    padded_spectrograms = []
    
    for element in list:
        padded = np.pad(element, [(0,0), (0, max_check - element[0].size)], mode = 'constant', constant_values = 0)
        padded = padded.reshape((padded.shape[0], padded.shape[1], 1))
        padded_spectrograms.append(padded)
        
    padded_spectrograms = np.array(padded_spectrograms)
    
    return padded_spectrograms

In [10]:
def data_loader():
    # fetches the current working directory
    path = os.getcwd()

    # stores the path for folder containing spectrogram arrays
    spectrogram_path = os.path.join(path,"train")
    # reads the csv folder containing the annotations
    label_chart = pd.read_csv(os.path.join(path, "annotations.csv"))
    # storing the label strings in one list
    labels = label_chart["fname"]


    # initialise variable that will store the list of the spectorgram arrays
    spectrograms = []
    
    # using a for loop to store all spectrogram files corresponding to a particular label
    for label in labels:
        spect = np.load(os.path.join(spectrogram_path, label))
        i,j,k = spect.shape
        spect.resize(j,k)
        mfcc = librosa.feature.mfcc(S = librosa.power_to_db(spect, ref=np.max))
        spectrograms.append(mfcc)
        
    padded_mfcc = padder(spectrograms)
    
    # encoding labels as one-hot vectors
    encoding = label_chart['label']
    encoder = LabelEncoder()
    encoded = encoder.fit_transform(encoding)
    Y = encoded
        
    # returns padded spectrograms and one-hot encoded labels     
    return padded_mfcc, Y

In [13]:
X_full, Y = data_loader()

In [14]:
X = []
for a in X_full:
    X.append(a.flatten())
X = np.array(X)

In [15]:
print(X.shape)

(1000, 51680)


In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=4)

In [38]:
grid_params = {
    'n_neighbors': [3, 4, 5, 6, 7],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'minkowski']
}

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

neigh = GridSearchCV(KNeighborsClassifier(), grid_params, cv=5, n_jobs=-1)
neigh.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['euclidean', 'minkowski'],
                         'n_neighbors': [3, 4, 5, 6, 7],
                         'weights': ['uniform', 'distance']})

In [39]:
print(neigh.best_params_)

{'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}


In [40]:
Y_pred = neigh.predict(X_test)

In [41]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_test, Y_pred)
cm

array([[12,  2,  0,  0,  1,  0,  0,  0,  3,  1],
       [ 0, 18,  0,  0,  0,  1,  0,  0,  3,  0],
       [ 0,  0, 14,  0,  0,  1,  0,  0,  0,  1],
       [ 1,  1,  0,  6,  2,  2,  1,  0,  5,  0],
       [ 0,  0,  0,  4, 11,  2,  1,  0,  2,  1],
       [ 2,  1,  2,  0,  1, 11,  0,  0,  1,  0],
       [ 1,  0,  5,  0,  2,  0,  9,  0,  2,  0],
       [ 0,  0,  0,  0,  1,  0,  0, 22,  0,  0],
       [ 0,  0,  2,  0,  0,  2,  1,  0, 18,  1],
       [ 0,  0,  0,  0,  2,  2,  1,  0,  5, 10]], dtype=int64)

In [42]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.75      0.63      0.69        19
           1       0.82      0.82      0.82        22
           2       0.61      0.88      0.72        16
           3       0.60      0.33      0.43        18
           4       0.55      0.52      0.54        21
           5       0.52      0.61      0.56        18
           6       0.69      0.47      0.56        19
           7       1.00      0.96      0.98        23
           8       0.46      0.75      0.57        24
           9       0.71      0.50      0.59        20

    accuracy                           0.66       200
   macro avg       0.67      0.65      0.65       200
weighted avg       0.68      0.66      0.65       200

